In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Colab Notebooks/春休み課題"

/content/drive/My Drive/Colab Notebooks/春休み課題


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import CIFAR10
%tensorflow_version 1.x
import tensorflow as tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/春休み課題
TensorFlow 1.x selected.


In [4]:
names, data, labels = CIFAR10.read_data('cifar-10-batches-py')

# 重みとバイアスの設定(ほんとは, モデルを作ってから書くとよい.)
x = tf.placeholder(tf.float32, [None, 24*24])
y = tf.placeholder(tf.float32, [None, len(names)])

w1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))

w2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))

w3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))

w_out = tf.Variable(tf.random_normal([1024, len(names)]))
b_out = tf.Variable(tf.random_normal([len(names)]))

names ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(50000, 3072) (50000,)


In [0]:
# 畳み込み層の作成
def conv_layer(x, W, b):
  conv = tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')
  conv_with_b = tf.nn.bias_add(conv, b)
  conv_out = tf.nn.relu(conv_with_b)
  return conv_out

In [0]:
# 最大プール層の作成
def maxpool_layer(conv, k=2):
  maxpool = tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
  return maxpool

In [0]:
# CNNモデル
def model():
  x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])

  #1層目: 畳み込み層
  conv_out1 = conv_layer(x_reshaped, w1, b1)
  maxpool_out1 = maxpool_layer(conv_out1) # 1枚が(12,12)
  norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75) #局所的応答正規化

  #2層目: 畳み込み層
  conv_out2 = conv_layer(norm1, w2, b2)
  maxpool_out2 = maxpool_layer(conv_out2) # 1枚が(6,6)
  norm2 = tf.nn.lrn(maxpool_out2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

  #3層目: 全結合層
  out_reshaped = tf.reshape(norm2, [-1, w3.get_shape().as_list()[0]])
  local = tf.add(tf.matmul(out_reshaped, w3), b3)
  local_out = tf.nn.relu(local)

  out = tf.add(tf.matmul(local_out, w_out), b_out)
  return out

**ソフトマックスクロスエントロピー**
1.   ソフトマックス関数
> 活性化関数としてよく用いられる. NNが出力する値:$x_{i}$を確率に変更する(つまり値を[0,1]の範囲に収め, 出力値の合計を1にする).多クラス分類のNNの出力をこの関数を通すことで, 分類の推定確率とできる.
$$softmax(x_{i}) = \frac{\exp(x_{i})}{\sum_{k=1}^{K}\exp{x_{k}}}$$


2.  クロスエントロピー
> 多クラス分類でよく使われる誤差関数の1つで, d番目のデータに関して, クラスiである推定確率$x_{d,i}$と真の確率$y_{d,i}$の誤差を計算する. NNでは一般的に, $x_{d,i}$にソフトマックス関数の出力を, $y_{d,i}$に教師データとする.
$$crossentoropy(x,y) = - \sum_{d=1}^{D}\sum_{i=1}^{K}log(x_{d,i})$$

3.  Tensorflowでは...
> **tf.nn.softmax_cross_entropy_with_logits_v2** を用いる.
クロスエントロピーの単体の関数はない. ソフトマックス関数とセット！
注意) クラスに対する和:$\sum_{i=1}^{K}$しか計算されないので, データの和:$\sum_{d=1}^{D}$はreduce_sumを用いて自分で計算すること.



**[用語確認]**
誤差関数 = 損失関数であり, コスト関数 = 誤差関数 + 正則化項


In [0]:
# 誤差関数(コスト関数,損失関数)と最適化の決定
model_op = model()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model_op, labels=y))
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1)) 
       # tf.argmax(y,1): y行列において,行ごとに最大となる列数を返す
       # tf.equal(x,y) : xとyの各要素を比較して正しいかを{True, False}で返す
acuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
       # tf.cast(x,a)  : x={True, False}をaの形式={0., 1.}で返す.
       # acuracyは結果的に, 正解割合を確率として表している

In [0]:
# 訓練
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  onehot_labels = tf.one_hot(indices=labels, depth=len(names), on_value=1., off_value=0., axis=-1)
      # indices : 実際に対応する(1を示す)番号の入ったベクトル
      #  depth  : 何クラスにするか
  onehot_vals = sess.run(onehot_labels)

  batch_size = len(data)

  for j in range(1000):
    print('EPOCH', j)
    for i in range(0, len(data), batch_size):
      batch_data = data[i:i+batch_size, :]
      batch_onehot_vals = onehot_vals[i:i+batch_size, :]
      _, accuracy_val = sess.run([train_op, acuracy], feed_dict={x:batch_data, y:batch_onehot_vals})
      if i% 1000 == 0:
        print(i, accuracy_val)
    print('DONE WITH EPOCH')

EPOCH 0
